In [ ]:
import numpy as np
import matplotlib.pylab as plt
import xml.etree.ElementTree as ET
%matplotlib notebook

import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
tree = ET.parse("quarter1.XML")
root = tree.getroot()
sequences = root[0][4][0][11]

In [ ]:
players = []
for i in range(0,11):
    players.append([])

for s in sequences:
    t = s.get('game-clock')
    locs = s.get('locations')
    
    locs = locs.split(';')
    
    for i in range(0,11):
        if len(locs)>10:
            vals = locs[i].split(',')
            players[i].append([int(vals[0]),int(vals[1]),float(vals[2]),float(vals[3]),float(vals[4]),float(t)])

In [ ]:
p = np.array(players)

In [ ]:
ball=p[0]

In [ ]:
x = ball.transpose()[2]
y = ball.transpose()[3]
z = ball.transpose()[4]
print(len(x))

In [ ]:
start = 0
stop = 1200

fig = plt.figure(figsize=(12,6))
ax = fig.gca(projection='3d')
ax.plot(x[start:stop], y[start:stop], z[start:stop], label='parametric curve',color='orange')


In [ ]:
plt.figure()
index1 = z>9.85
index2 = z<10.15

index = index1*index2

print(index)
plt.plot(x[index],y[index],'ko')

In [ ]:
# Find all the unique players

# Team 1
ids1 = []
for i in range(1,6):
    ids1 += p[i].transpose()[1].astype(int).tolist()
    
# Team 2
ids2 = []
for i in range(6,11):
    ids2 += p[i].transpose()[1].astype(int).tolist()    

In [ ]:
print(len(ids1))
team1 = np.unique(ids1)
print(team1)

print(len(ids2))
team2 = np.unique(ids2)
print(team2)


In [ ]:
player_mapping = np.loadtxt('player_ids.csv',unpack=True,skiprows=1,dtype=bytes,delimiter=',')
print(player_mapping[0][0:10])
print(player_mapping[1][0:10])

In [ ]:
print("Team 1")
print("----------")
for ids in team1:
    name = player_mapping[1][player_mapping[0].astype(float).tolist().index(ids)]
    print(name.decode())
    
print("\nTeam 2")
print("----------")
for ids in team2:
    name = player_mapping[1][player_mapping[0].astype(float).tolist().index(ids)]
    print(name.decode())
    


In [ ]:
def pid2name(pid,player_mapping):
    name = player_mapping[1][player_mapping[0].astype(int).tolist().index(pid)]
    return name.decode()

print(pid2name(3234,player_mapping))


In [ ]:
team1_players = []
for i in range(0,len(team1)):
    team1_players.append([])
    for k in range(0,6):
        team1_players[i].append([])
    
    for j in range(1,6):
        #team1_players[i].append()
        pid = p[j].transpose()[1].astype(int)
        x = p[j].transpose()[2].astype(float)
        y = p[j].transpose()[3].astype(float)
        z = p[j].transpose()[4].astype(float)
        t = 720 - p[j].transpose()[5].astype(float)

        index = team1[i] == pid
        
        team1_players[i][0] += pid[index].tolist()
        team1_players[i][1] += x[index].tolist()
        team1_players[i][2] += y[index].tolist()
        team1_players[i][3] += z[index].tolist()
        team1_players[i][4] += t[index].tolist()

    # Do this to sort everyone, according to the last entry, time
    sort_index = np.argsort(team1_players[i][4])
    
    for k in range(0,5):
        team1_players[i][k] = np.array(team1_players[i][k])[sort_index]
    
        
        

In [ ]:
team1_players[0]

In [ ]:
start = 0
stop = 1200

fig = plt.figure(figsize=(12,6))
ax = fig.gca(projection='3d')
x = team1_players[0][1]
y = team1_players[0][2]
z = team1_players[0][3]

ax.plot(x[start:stop], y[start:stop], z[start:stop], '.', label='Dirk',color='blue',markersize=1)


In [ ]:
fig = plt.figure(figsize=(10,5))
x = team1_players[0][1]
y = team1_players[0][2]
z = team1_players[0][3]
t = team1_players[0][4]


plt.plot(t, x,'.')
plt.xlim(0,720)

In [ ]:
# Add in velocities
for i in range(0,len(team1_players)):
    t = team1_players[i][4]
    x = team1_players[i][1]
    y = team1_players[i][2]
    z = team1_players[i][3]
    v = np.zeros(len(team1_players[i][4]))

    dt = t[2:] - t[0:-2]
    dx = x[2:] - x[0:-2]
    dy = y[2:] - y[0:-2]
    dz = z[2:] - z[0:-2]
    
    dist = np.sqrt(dx*dx + dy*dy + dz*dz)
    
    v[1:-1] = dist/dt
    
    # Remove excessively high speeds
    v[v>20] = 0.0
    
    team1_players[i][5] = v

In [ ]:
plt.figure()
t = team1_players[0][4]
v = team1_players[0][5]
print(len(team1_players[0]),len(t),len(v))
print(v)
plt.plot(t,v)
plt.xlim(0,720)

In [ ]:
plt.figure(figsize=(10,5))

for i in range(len(team1_players)):
    
    pid = team1_players[i][0][0]
    
    name = pid2name(pid,player_mapping)
    
    print(pid,name)
    
    pcts = []
    cutoffs = np.arange(0,22,0.1)
    for cutoff in cutoffs:
        v = team1_players[i][5]
        tot = len(v[v>0])

        pct = len(v[v>cutoff])/float(tot)

        pcts.append(pct)

    plt.plot(cutoffs,pcts,label=name,linewidth=3)
plt.legend()